In [80]:
import numpy as np
import pandas as pd

def generate_data_with_features(samples=1000, seconds=60):
    np.random.seed(42)
    
    rolling_data = []
    pitching_data = []
    stress_values = []

    for _ in range(samples):
        rolling_sample = np.random.uniform(-0.6, 0.6, seconds)
        pitching_sample = np.random.uniform(-1.0, 1.0, seconds)
        
        rolling_mean_change = np.mean(np.abs(np.diff(rolling_sample)))
        pitching_mean_change = np.mean(np.abs(np.diff(pitching_sample)))
        
        stress = rolling_mean_change * 5 + pitching_mean_change * 5  # Weighted sum
        
        stress = max(0, min(10, stress))
        
        rolling_data.append(rolling_sample)
        pitching_data.append(pitching_sample)
        stress_values.append(stress)

    rolling_columns = [f'rolling_{i+1}' for i in range(seconds)]
    pitching_columns = [f'pitching_{i+1}' for i in range(seconds)]
    data = pd.DataFrame(rolling_data, columns=rolling_columns)
    data = pd.concat([data, pd.DataFrame(pitching_data, columns=pitching_columns)], axis=1)
    data['stress'] = stress_values
    
    return data

data = generate_data_with_features(samples=1000, seconds=60)
print("Sample Data:")
print(data.head())


Sample Data:
   rolling_1  rolling_2  rolling_3  rolling_4  rolling_5  rolling_6  \
0  -0.150552   0.540857   0.278393   0.118390  -0.412778  -0.412807   
1   0.368928   0.475310  -0.218396  -0.467938  -0.326478  -0.087471   
2   0.528550   0.544714   0.497837  -0.155810  -0.581452   0.513982   
3  -0.134196   0.171946  -0.050097   0.054740   0.529758  -0.136677   
4   0.155278   0.234898  -0.054551   0.153070   0.101177   0.481390   

   rolling_7  rolling_8  rolling_9  rolling_10  ...  pitching_52  pitching_53  \
0  -0.530300   0.439411   0.121338    0.249687  ...    -0.677557     0.859395   
1   0.381618   0.432877  -0.591657    0.012897  ...     0.005274     0.153808   
2  -0.086179   0.559986   0.556344    0.423611  ...     0.712980     0.317387   
3   0.553429   0.486421  -0.365051   -0.516766  ...     0.939758    -0.913680   
4  -0.545464  -0.262844   0.540494    0.468317  ...     0.281749    -0.636240   

   pitching_54  pitching_55  pitching_56  pitching_57  pitching_58  \
0  

In [81]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def preprocess_and_extract_features(data):
    features = pd.DataFrame()
    seconds = 60

    rolling_cols = [f'rolling_{i+1}' for i in range(seconds)]
    pitching_cols = [f'pitching_{i+1}' for i in range(seconds)]
    
    features['rolling_mean_change'] = data[rolling_cols].diff(axis=1).abs().mean(axis=1)
    features['rolling_std'] = data[rolling_cols].std(axis=1)
    features['pitching_mean_change'] = data[pitching_cols].diff(axis=1).abs().mean(axis=1)
    features['pitching_std'] = data[pitching_cols].std(axis=1)
    
    y = data['stress']
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler

X_train, X_test, y_train, y_test, scaler = preprocess_and_extract_features(data)


In [83]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest(X_train, y_train, n_estimators=100):
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    model.fit(X_train, y_train)
    return model

model = train_random_forest(X_train, y_train, n_estimators=200)


In [84]:
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return {'mse': mse, 'r2': r2, 'predictions': y_pred}

metrics = evaluate_model(model, X_test, y_test)
print(f"MSE: {metrics['mse']:.4f}")
print(f"R2 Score: {metrics['r2']:.4f}")


MSE: 0.0022
R2 Score: 0.9849


In [85]:
def predict_stress(model, scaler, rolling_data, pitching_data):
    features = {}
    features['rolling_mean_change'] = np.mean(np.abs(np.diff(rolling_data)))
    features['rolling_std'] = np.std(rolling_data)
    features['pitching_mean_change'] = np.mean(np.abs(np.diff(pitching_data)))
    features['pitching_std'] = np.std(pitching_data)
    
    input_df = pd.DataFrame([features])
    input_scaled = scaler.transform(input_df)
    
    stress_prediction = model.predict(input_scaled)
    return stress_prediction[0]

user_rolling = np.random.uniform(-0.6, 0.6, 60).tolist()
user_pitching = np.random.uniform(-1.0, 1.0, 60).tolist()

predicted_stress = predict_stress(model, scaler, user_rolling, user_pitching)
print(f"Predicted Stress Level: {predicted_stress:.2f}")


Predicted Stress Level: 4.72


In [86]:
import joblib

def save_model(model, scaler, model_path="stress_model.pkl", scaler_path="scaler.pkl"):
    joblib.dump(model, model_path)
    joblib.dump(scaler, scaler_path)
    print(f"Model saved to {model_path}")
    print(f"Scaler saved to {scaler_path}")

save_model(model, scaler)


Model saved to stress_model.pkl
Scaler saved to scaler.pkl
